In [1]:
import numpy as  np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import os
import glob
import itertools
from tqdm import tqdm


In [2]:
# Get list of sheets within excel file
def get_sheet_names(file_path):
    xl = pd.ExcelFile(file_path)
    return xl.sheet_names

get_sheet_names(r'Final Master Data.xlsx')

['Sampling Cycles',
 'Meterological Data',
 'PM Data',
 'LAMPS FT Raw Data',
 'LAMPS DN Raw Data',
 'CampSci Reference Panel',
 'Day Fraction',
 'LAMPS Calculated Soiling Loss',
 'Additional Data']

In [3]:
Data = {}

In [4]:
def Meteorological_Data():
    # Read sheet name 'Meterological Data'
    df = pd.read_excel(r'Final Master Data.xlsx', sheet_name='Meterological Data', skiprows=0)

    # Drop column RECORD
    df.drop('RECORD', axis=1, inplace=True)

    # Drop rows with all NaN values
    df.dropna(axis=0, how='all', inplace=True)

    # Merge Date and Time columns
    df['DateTime'] = pd.to_datetime(df['Date'].astype('str') + ' ' + df['Time'].astype('str'))

    # Drop Date and Time columns
    df.drop(['Date', 'Time'], axis=1, inplace=True)

    # Only keep rows with DateTime values between 2022-01-10 and 2022-04-19 (inclusive)
    df = df[(df['DateTime'] >= '2022-01-10T00:00:00') & (df['DateTime'] <= '2022-04-20T00:00:00')]

    # Resample to minute frequency and interpolate missing values
    df = df.resample('T', on='DateTime').mean().interpolate()

    df = df.iloc[:-1, :]

    return df

Data['Meteorological_Data'] = Meteorological_Data()

In [5]:
def PM_Data():
    # Read sheet name 'PM Data'
    df = pd.read_excel(r'Final Master Data.xlsx', sheet_name='PM Data', skiprows=0)

    # Drop rows with all NaN values
    df.dropna(axis=0, how='all', inplace=True)

    # Strip whitespace from column names
    df.columns = df.columns.str.strip()

    # Rename column 'yyyy/mm/dd' to 'Date'
    df.rename(columns={'yyyy/mm/dd': 'Date'}, inplace=True)

    # Rename column 'hh:mm:ss' to 'Time'
    df.rename(columns={'hh:mm:ss': 'Time'}, inplace=True)

    # Merge Date and Time columns
    df['DateTime'] = pd.to_datetime(df['Date'].astype('str') + ' ' + df['Time'].astype('str'))

    # Drop Date and Time columns
    df.drop(['Date', 'Time'], axis=1, inplace=True)

    # Only keep rows with DateTime values between 2022-01-10 and 2022-04-19 (inclusive)
    df = df[(df['DateTime'] >= '2022-01-10T00:00:00') & (df['DateTime'] <= '2022-04-20T00:00:00')]

    # Resample to minute frequency and interpolate missing values
    df = df.resample('T', on='DateTime').mean().interpolate()

    return df

Data['PM_Data'] = PM_Data()

In [6]:
def CampSci_Reference_Panel():
    # Read sheet name 'CampSci Reference Panel'
    df = pd.read_excel(r'Final Master Data.xlsx', sheet_name='CampSci Reference Panel', skiprows=1)

    # Drop row index 1
    df.drop(0, axis=0, inplace=True)

    # Drop rows with all NaN values
    df.dropna(axis=0, how='all', inplace=True)

    # Strip whitespace from column names
    df.columns = df.columns.str.strip()

    # Rename column 'TIMESTAMP' to 'DateTime'
    df.rename(columns={'TIMESTAMP': 'DateTime'}, inplace=True)

    # Set 'DateTime' column as datetime
    df['DateTime'] = pd.to_datetime(df['DateTime'])

    # Set columns 'GeffRef' and 'TempRef' as float
    df['GeffRef'] = df['GeffRef'].astype('float')
    df['TempRef'] = df['TempRef'].astype('float')

    # Set columns 'GeffRef' and 'TempRef' negative values to 0
    df['GeffRef'] = df['GeffRef'].clip(lower=0)

    # Only keep rows with DateTime values between 2022-01-10 and 2022-04-19 (inclusive)
    df = df[(df['DateTime'] >= '2022-01-10T00:00:00') & (df['DateTime'] <= '2022-04-20T00:00:00')]

    # Resample to minute frequency and interpolate missing values
    df = df.resample('T', on='DateTime').mean().interpolate()

    df = df.iloc[:-1, :]

    return df

Data['CampSci_Reference_Panel'] = CampSci_Reference_Panel()

In [9]:
# Merge Data values for keys except Sampling_Cycles
combined_df = pd.concat([Data['Meteorological_Data'], Data['PM_Data'], Data['CampSci_Reference_Panel']], axis=1)
combined_df

,PTemp_C_Max,AirTC_Avg,RH,WS_ms_Avg,WindDir,TPM1,TPM2.5,TPM10,GeffRef,TempRef
DateTime,,,,,,,,,,
2022-01-10 00:00:00,14.130,14.170,70.200,0.0810,252.700,42.0,62.0,70.0,0.000559,11.0750
2022-01-10 00:01:00,14.116,14.150,70.322,0.0742,215.844,42.0,62.0,67.0,0.001397,11.0625
2022-01-10 00:02:00,14.102,14.130,70.444,0.0674,178.988,44.0,61.0,64.0,0.002235,11.0500
2022-01-10 00:03:00,14.088,14.110,70.566,0.0606,142.132,45.0,64.0,68.0,0.006148,11.0250
2022-01-10 00:04:00,14.074,14.090,70.688,0.0538,105.276,44.0,63.0,67.0,0.000559,10.9900
...,...,...,...,...,...,...,...,...,...,...
2022-04-19 23:55:00,31.690,29.340,41.480,0.7820,156.400,37.0,55.0,61.0,0.036891,27.9250
2022-04-19 23:56:00,31.646,29.482,41.006,0.6770,149.340,37.0,55.0,61.0,0.001537,27.9550
2022-04-19 23:57:00,31.602,29.624,40.532,0.5720,142.280,33.0,49.0,53.0,0.085030,27.9800


In [27]:
def get_Sampling_Cycles():
    # Read sheet name 'Sampling Cycles'
    df = pd.read_excel(r'Final Master Data.xlsx', sheet_name='Sampling Cycles', skiprows=1)

    # Drop Unnamed: 0 column
    df.drop('Unnamed: 0', axis=1, inplace=True)

    # Drop rows with all NaN values
    df.dropna(axis=0, how='all', inplace=True)

    # Keep rows with Cycle values [3,4,5,6,7]
    df = df[df['Cycle'].isin([3,4,5,6,7])]

    # Print number of ones in column Rainfall
    print(df['Rainfall'].value_counts())

    # Print first and last Date
    print(df['Date'].iloc[0], df['Date'].iloc[-1])

    # Reset index
    df.reset_index(drop=True, inplace=True)

    return df

# Data['Sampling_Cycles'] turn from daily to minute data by repeating columns 'Cycle' and 'Rainfall' for entire day
# Data['Sampling_Cycles'] = 
# Data['Sampling_Cycles'].loc[np.repeat(Data['Sampling_Cycles'].index.values, 1440)]

Data['Sampling_Cycles'] = get_Sampling_Cycles()
Data['Sampling_Cycles']

Rainfall
0    100
Name: count, dtype: int64
2022-01-10 00:00:00 2022-04-19 00:00:00


,Date,Cycle,Rainfall
0,2022-01-10,3,0
1,2022-01-11,3,0
2,2022-01-12,3,0
3,2022-01-13,3,0
4,2022-01-14,3,0
...,...,...,...
95,2022-04-15,7,0
96,2022-04-16,7,0
97,2022-04-17,7,0
98,2022-04-18,7,0


In [44]:
# Create new column 'Cycle' in combined_df
combined_df['Cycle'] = np.nan
for row in tqdm(range(combined_df.shape[0])):
    cycle_number = Data['Sampling_Cycles'][Data['Sampling_Cycles']['Date'] == str(combined_df.index[row].date())]['Cycle'].values[0]
    combined_df['Cycle'].iloc[row] = cycle_number

100%|██████████| 144000/144000 [01:05<00:00, 2210.96it/s]


In [45]:
combined_df

,PTemp_C_Max,AirTC_Avg,RH,WS_ms_Avg,WindDir,TPM1,TPM2.5,TPM10,GeffRef,TempRef,Cycle
DateTime,,,,,,,,,,,
2022-01-10 00:00:00,14.130,14.170,70.200,0.0810,252.700,42.0,62.0,70.0,0.000559,11.0750,3.0
2022-01-10 00:01:00,14.116,14.150,70.322,0.0742,215.844,42.0,62.0,67.0,0.001397,11.0625,3.0
2022-01-10 00:02:00,14.102,14.130,70.444,0.0674,178.988,44.0,61.0,64.0,0.002235,11.0500,3.0
2022-01-10 00:03:00,14.088,14.110,70.566,0.0606,142.132,45.0,64.0,68.0,0.006148,11.0250,3.0
2022-01-10 00:04:00,14.074,14.090,70.688,0.0538,105.276,44.0,63.0,67.0,0.000559,10.9900,3.0
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-19 23:55:00,31.690,29.340,41.480,0.7820,156.400,37.0,55.0,61.0,0.036891,27.9250,7.0
2022-04-19 23:56:00,31.646,29.482,41.006,0.6770,149.340,37.0,55.0,61.0,0.001537,27.9550,7.0
2022-04-19 23:57:00,31.602,29.624,40.532,0.5720,142.280,33.0,49.0,53.0,0.085030,27.9800,7.0


In [46]:
# Create new column 'Rainfall' in combined_df
combined_df['Rainfall'] = np.nan
for row in tqdm(range(combined_df.shape[0])):
    rainfall = Data['Sampling_Cycles'][Data['Sampling_Cycles']['Date'] == str(combined_df.index[row].date())]['Rainfall'].values[0]
    combined_df['Rainfall'].iloc[row] = rainfall

100%|██████████| 144000/144000 [01:05<00:00, 2199.16it/s]


In [47]:
combined_df

,PTemp_C_Max,AirTC_Avg,RH,WS_ms_Avg,WindDir,TPM1,TPM2.5,TPM10,GeffRef,TempRef,Cycle,Rainfall
DateTime,,,,,,,,,,,,
2022-01-10 00:00:00,14.130,14.170,70.200,0.0810,252.700,42.0,62.0,70.0,0.000559,11.0750,3.0,0.0
2022-01-10 00:01:00,14.116,14.150,70.322,0.0742,215.844,42.0,62.0,67.0,0.001397,11.0625,3.0,0.0
2022-01-10 00:02:00,14.102,14.130,70.444,0.0674,178.988,44.0,61.0,64.0,0.002235,11.0500,3.0,0.0
2022-01-10 00:03:00,14.088,14.110,70.566,0.0606,142.132,45.0,64.0,68.0,0.006148,11.0250,3.0,0.0
2022-01-10 00:04:00,14.074,14.090,70.688,0.0538,105.276,44.0,63.0,67.0,0.000559,10.9900,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-19 23:55:00,31.690,29.340,41.480,0.7820,156.400,37.0,55.0,61.0,0.036891,27.9250,7.0,0.0
2022-04-19 23:56:00,31.646,29.482,41.006,0.6770,149.340,37.0,55.0,61.0,0.001537,27.9550,7.0,0.0
2022-04-19 23:57:00,31.602,29.624,40.532,0.5720,142.280,33.0,49.0,53.0,0.085030,27.9800,7.0,0.0


In [48]:
# Check if combined_df has any NaN values
combined_df.isna().sum()

PTemp_C_Max    0
AirTC_Avg      0
RH             0
WS_ms_Avg      0
WindDir        0
TPM1           0
TPM2.5         0
TPM10          0
GeffRef        0
TempRef        0
Cycle          0
Rainfall       0
dtype: int64

In [51]:
sunrise_sunset = pd.read_excel('Sunrise and Sunset Master.xlsx', skiprows=1)
sunrise_sunset.drop('Unnamed: 0', axis=1, inplace=True)
sunrise_sunset

,Date,Sunrise,Sunset
0,2021-12-01,07:03:00,17:52:00
1,2021-12-02,07:04:00,17:52:00
2,2021-12-03,07:05:00,17:52:00
3,2021-12-04,07:05:00,17:53:00
4,2021-12-05,07:06:00,17:53:00
...,...,...,...
175,2022-05-25,05:54:00,19:18:00
176,2022-05-26,05:54:00,19:18:00
177,2022-05-27,05:54:00,19:19:00
178,2022-05-28,05:53:00,19:19:00


In [60]:
combined_df['Daylight'] = np.nan
for row in tqdm(range(combined_df.shape[0])):
    index_date = combined_df.index[row].date()
    index_time = combined_df.index[row].time()
    sunrise_time = sunrise_sunset[sunrise_sunset['Date'] == str(index_date)]['Sunrise'].values[0]
    sunset_time = sunrise_sunset[sunrise_sunset['Date'] == str(index_date)]['Sunset'].values[0]
    # print(sunrise_time, sunset_time, index_date)
    # print(combined_df.index[row].time() >= sunrise_time and combined_df.index[row].time() <= sunset_time)
    # print(combined_df.index[row])

    if combined_df.index[row].time() >= sunrise_time and index_time <= sunset_time:
        combined_df['Daylight'].iloc[row] = 1
    else:
        combined_df['Daylight'].iloc[row] = 0
combined_df

100%|██████████| 144000/144000 [01:45<00:00, 1363.33it/s]


,PTemp_C_Max,AirTC_Avg,RH,WS_ms_Avg,WindDir,TPM1,TPM2.5,TPM10,GeffRef,TempRef,Cycle,Rainfall,Daylight
DateTime,,,,,,,,,,,,,
2022-01-10 00:00:00,14.130,14.170,70.200,0.0810,252.700,42.0,62.0,70.0,0.000559,11.0750,3.0,0.0,0.0
2022-01-10 00:01:00,14.116,14.150,70.322,0.0742,215.844,42.0,62.0,67.0,0.001397,11.0625,3.0,0.0,0.0
2022-01-10 00:02:00,14.102,14.130,70.444,0.0674,178.988,44.0,61.0,64.0,0.002235,11.0500,3.0,0.0,0.0
2022-01-10 00:03:00,14.088,14.110,70.566,0.0606,142.132,45.0,64.0,68.0,0.006148,11.0250,3.0,0.0,0.0
2022-01-10 00:04:00,14.074,14.090,70.688,0.0538,105.276,44.0,63.0,67.0,0.000559,10.9900,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-19 23:55:00,31.690,29.340,41.480,0.7820,156.400,37.0,55.0,61.0,0.036891,27.9250,7.0,0.0,0.0
2022-04-19 23:56:00,31.646,29.482,41.006,0.6770,149.340,37.0,55.0,61.0,0.001537,27.9550,7.0,0.0,0.0
2022-04-19 23:57:00,31.602,29.624,40.532,0.5720,142.280,33.0,49.0,53.0,0.085030,27.9800,7.0,0.0,0.0


In [61]:
# Give count of 1 and 0 in column 'Daylight'
combined_df['Daylight'].value_counts()

Daylight
0.0    73634
1.0    70366
Name: count, dtype: int64

In [62]:
# Save combined_df to csv
combined_df.to_csv('Combined_Data.csv')

# Add FT and DN data to the combined data

In [2]:
# Load combined_df from csv
combined_df = pd.read_csv('Combined_Data.csv', index_col=0)
combined_df

,PTemp_C_Max,AirTC_Avg,RH,WS_ms_Avg,WindDir,TPM1,TPM2.5,TPM10,GeffRef,TempRef,Cycle,Rainfall,Daylight
DateTime,,,,,,,,,,,,,
2022-01-10 00:00:00,14.130,14.170,70.200,0.0810,252.700,42.0,62.0,70.0,0.000559,11.0750,3.0,0.0,0.0
2022-01-10 00:01:00,14.116,14.150,70.322,0.0742,215.844,42.0,62.0,67.0,0.001397,11.0625,3.0,0.0,0.0
2022-01-10 00:02:00,14.102,14.130,70.444,0.0674,178.988,44.0,61.0,64.0,0.002235,11.0500,3.0,0.0,0.0
2022-01-10 00:03:00,14.088,14.110,70.566,0.0606,142.132,45.0,64.0,68.0,0.006148,11.0250,3.0,0.0,0.0
2022-01-10 00:04:00,14.074,14.090,70.688,0.0538,105.276,44.0,63.0,67.0,0.000559,10.9900,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-19 23:55:00,31.690,29.340,41.480,0.7820,156.400,37.0,55.0,61.0,0.036891,27.9250,7.0,0.0,0.0
2022-04-19 23:56:00,31.646,29.482,41.006,0.6770,149.340,37.0,55.0,61.0,0.001537,27.9550,7.0,0.0,0.0
2022-04-19 23:57:00,31.602,29.624,40.532,0.5720,142.280,33.0,49.0,53.0,0.085030,27.9800,7.0,0.0,0.0


In [7]:
def LAMPS_FT():
    # Read sheet name 'LAMPS FT Raw Data'
    df = pd.read_excel(r'Final Master Data.xlsx', sheet_name='LAMPS FT Raw Data', skiprows=0)

    # Drop rows with all NaN values
    df.dropna(axis=0, how='all', inplace=True)

    # Strip whitespace from column names
    df.columns = df.columns.str.strip()

    # Drop column 'millis'
    df.drop('millis', axis=1, inplace=True)

    # Set 'Date' column as datetime
    df.drop('Date', axis=1, inplace=True)
    df['Date'] = pd.to_datetime(np.load('FT_Dates.npy'))

    # # Merge Date and Time columns
    df['DateTime'] = pd.to_datetime(df['Date'].astype('str') + ' ' + df['Time'].astype('str'))

    # # Drop Date and Time columns
    df.drop(['Date', 'Time'], axis=1, inplace=True)

    # Only keep rows with DateTime values between 2022-01-10 and 2022-04-19 (inclusive)
    df = df[(df['DateTime'] >= '2022-01-10T00:00:00') & (df['DateTime'] <= '2022-04-20T00:00:00')]

    # Resample to minute frequency and interpolate missing values
    # df = df.resample('T', on='DateTime').mean().interpolate()

    return df

Data_LAMPS_FT = LAMPS_FT()
Data_LAMPS_FT

,voltage1,voltage2,DateTime
57986,0.02,0.02,2022-01-10 00:00:41
57987,0.02,0.02,2022-01-10 00:01:40
57988,0.02,0.02,2022-01-10 00:02:39
57989,0.02,0.02,2022-01-10 00:03:38
57990,0.02,0.02,2022-01-10 00:04:37
...,...,...,...
203393,0.02,0.02,2022-04-19 23:55:37
203394,0.02,0.02,2022-04-19 23:56:36
203395,0.02,0.02,2022-04-19 23:57:35
203396,0.02,0.02,2022-04-19 23:58:34


In [8]:
def LAMPS_DN():
    # Read sheet name 'LAMPS DN Raw Data'
    df = pd.read_excel(r'Final Master Data.xlsx', sheet_name='LAMPS DN Raw Data', skiprows=0)

    # Drop rows with all NaN values
    df.dropna(axis=0, how='all', inplace=True)

    # Strip whitespace from column names
    df.columns = df.columns.str.strip()

    # Drop column 'millis'
    df.drop('millis', axis=1, inplace=True)

    # Set 'Date' column as datetime
    df.drop('Date', axis=1, inplace=True)
    df['Date'] = pd.to_datetime(np.load('DN_Dates.npy'))

    # # Merge Date and Time columns
    df['DateTime'] = pd.to_datetime(df['Date'].astype('str') + ' ' + df['Time'].astype('str'))

    # # Drop Date and Time columns
    df.drop(['Date', 'Time'], axis=1, inplace=True)

    # Only keep rows with DateTime values between 2022-01-10 and 2022-04-19 (inclusive)
    df = df[(df['DateTime'] >= '2022-01-10T00:00:00') & (df['DateTime'] <= '2022-04-20T00:00:00')]

    # Resample to minute frequency and interpolate missing values
    # df = df.resample('T', on='DateTime').mean().interpolate()

    return df

Data_LAMPS_DN = LAMPS_DN()
Data_LAMPS_DN

,voltage1,voltage2,DateTime
58920,0.00,0.00,2022-01-10 00:00:42
58921,0.00,0.00,2022-01-10 00:01:41
58922,0.00,0.00,2022-01-10 00:02:40
58923,0.00,0.00,2022-01-10 00:03:39
58924,0.00,0.00,2022-01-10 00:04:38
...,...,...,...
204323,0.02,0.02,2022-04-19 23:56:01
204324,0.02,0.02,2022-04-19 23:57:00
204325,0.02,0.02,2022-04-19 23:57:59
204326,0.02,0.02,2022-04-19 23:58:58


In [ ]:
Data_LAMPS_FT.reset_index(drop=True, inplace=True)
Data_LAMPS_DN.reset_index(drop=True, inplace=True)

In [9]:
# Use df['DateTime'].dt.floor('T') on Data_LAMPS_FT and Data_LAMPS_DN to floor DateTime to minute frequency
Data_LAMPS_FT['DateTime'] = Data_LAMPS_FT['DateTime'].dt.floor('T')
Data_LAMPS_DN['DateTime'] = Data_LAMPS_DN['DateTime'].dt.floor('T')

In [10]:
# Set 0.02 or less to 0 in column voltage1 and voltage2 for both Data_LAMPS_FT and Data_LAMPS_DN
Data_LAMPS_FT['voltage1'] = Data_LAMPS_FT['voltage1'].apply(lambda x: 0 if x <= 0.02 else x)
Data_LAMPS_FT['voltage2'] = Data_LAMPS_FT['voltage2'].apply(lambda x: 0 if x <= 0.02 else x)

Data_LAMPS_DN['voltage1'] = Data_LAMPS_DN['voltage1'].apply(lambda x: 0 if x <= 0.02 else x)
Data_LAMPS_DN['voltage2'] = Data_LAMPS_DN['voltage2'].apply(lambda x: 0 if x <= 0.02 else x)

In [17]:
# Take mean when there are multiple values for the same DateTime
Data_LAMPS_FT = Data_LAMPS_FT.groupby('DateTime').mean()
Data_LAMPS_DN = Data_LAMPS_DN.groupby('DateTime').mean()
print(Data_LAMPS_FT.shape, Data_LAMPS_DN.shape)

(142994, 2) (142986, 2)


In [18]:
Data_LAMPS_FT

,voltage1,voltage2
DateTime,,
2022-01-10 00:00:00,0.0,0.0
2022-01-10 00:01:00,0.0,0.0
2022-01-10 00:02:00,0.0,0.0
2022-01-10 00:03:00,0.0,0.0
2022-01-10 00:04:00,0.0,0.0
...,...,...
2022-04-19 23:55:00,0.0,0.0
2022-04-19 23:56:00,0.0,0.0
2022-04-19 23:57:00,0.0,0.0


In [19]:
Data_LAMPS_DN

,voltage1,voltage2
DateTime,,
2022-01-10 00:00:00,0.0,0.0
2022-01-10 00:01:00,0.0,0.0
2022-01-10 00:02:00,0.0,0.0
2022-01-10 00:03:00,0.0,0.0
2022-01-10 00:04:00,0.0,0.0
...,...,...
2022-04-19 23:55:00,0.0,0.0
2022-04-19 23:56:00,0.0,0.0
2022-04-19 23:57:00,0.0,0.0


In [21]:
combined_df['FT_Voltage1'] = np.nan
combined_df['FT_Voltage2'] = np.nan
combined_df['DN_Voltage1'] = np.nan
combined_df['DN_Voltage2'] = np.nan
combined_df

,PTemp_C_Max,AirTC_Avg,RH,WS_ms_Avg,WindDir,TPM1,TPM2.5,TPM10,GeffRef,TempRef,Cycle,Rainfall,Daylight,FT_Voltage1,FT_Voltage2,DN_Voltage1,DN_Voltage2
DateTime,,,,,,,,,,,,,,,,,
2022-01-10 00:00:00,14.130,14.170,70.200,0.0810,252.700,42.0,62.0,70.0,0.000559,11.0750,3.0,0.0,0.0,NaN,NaN,NaN,NaN
2022-01-10 00:01:00,14.116,14.150,70.322,0.0742,215.844,42.0,62.0,67.0,0.001397,11.0625,3.0,0.0,0.0,NaN,NaN,NaN,NaN
2022-01-10 00:02:00,14.102,14.130,70.444,0.0674,178.988,44.0,61.0,64.0,0.002235,11.0500,3.0,0.0,0.0,NaN,NaN,NaN,NaN
2022-01-10 00:03:00,14.088,14.110,70.566,0.0606,142.132,45.0,64.0,68.0,0.006148,11.0250,3.0,0.0,0.0,NaN,NaN,NaN,NaN
2022-01-10 00:04:00,14.074,14.090,70.688,0.0538,105.276,44.0,63.0,67.0,0.000559,10.9900,3.0,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-19 23:55:00,31.690,29.340,41.480,0.7820,156.400,37.0,55.0,61.0,0.036891,27.9250,7.0,0.0,0.0,NaN,NaN,NaN,NaN
2022-04-19 23:56:00,31.646,29.482,41.006,0.6770,149.340,37.0,55.0,61.0,0.001537,27.9550,7.0,0.0,0.0,NaN,NaN,NaN,NaN
2022-04-19 23:57:00,31.602,29.624,40.532,0.5720,142.280,33.0,49.0,53.0,0.085030,27.9800,7.0,0.0,0.0,NaN,NaN,NaN,NaN


In [27]:
# Set values in column 'FT_Voltage1' and 'FT_Voltage2' in combined_df to values in Data_LAMPS_FT
# Set values in column 'DN_Voltage1' and 'DN_Voltage2' in combined_df to values in Data_LAMPS_DN
for row in tqdm(range(len(combined_df))):
    if combined_df.index[row] in Data_LAMPS_FT.index:
        combined_df['FT_Voltage1'].iloc[row] = Data_LAMPS_FT['voltage1'].loc[combined_df.index[row]]
        combined_df['FT_Voltage2'].iloc[row] = Data_LAMPS_FT['voltage2'].loc[combined_df.index[row]]

    if combined_df.index[row] in Data_LAMPS_DN.index:
        combined_df['DN_Voltage1'].iloc[row] = Data_LAMPS_DN['voltage1'].loc[combined_df.index[row]]
        combined_df['DN_Voltage2'].iloc[row] = Data_LAMPS_DN['voltage2'].loc[combined_df.index[row]]

combined_df

100%|██████████| 144000/144000 [02:02<00:00, 1173.39it/s]


,PTemp_C_Max,AirTC_Avg,RH,WS_ms_Avg,WindDir,TPM1,TPM2.5,TPM10,GeffRef,TempRef,Cycle,Rainfall,Daylight,FT_Voltage1,FT_Voltage2,DN_Voltage1,DN_Voltage2
DateTime,,,,,,,,,,,,,,,,,
2022-01-10 00:00:00,14.130,14.170,70.200,0.0810,252.700,42.0,62.0,70.0,0.000559,11.0750,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-10 00:01:00,14.116,14.150,70.322,0.0742,215.844,42.0,62.0,67.0,0.001397,11.0625,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-10 00:02:00,14.102,14.130,70.444,0.0674,178.988,44.0,61.0,64.0,0.002235,11.0500,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-10 00:03:00,14.088,14.110,70.566,0.0606,142.132,45.0,64.0,68.0,0.006148,11.0250,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-10 00:04:00,14.074,14.090,70.688,0.0538,105.276,44.0,63.0,67.0,0.000559,10.9900,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-19 23:55:00,31.690,29.340,41.480,0.7820,156.400,37.0,55.0,61.0,0.036891,27.9250,7.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-19 23:56:00,31.646,29.482,41.006,0.6770,149.340,37.0,55.0,61.0,0.001537,27.9550,7.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-19 23:57:00,31.602,29.624,40.532,0.5720,142.280,33.0,49.0,53.0,0.085030,27.9800,7.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# Check if combined_df has any NaN values
combined_df.isna().sum()

PTemp_C_Max       0
AirTC_Avg         0
RH                0
WS_ms_Avg         0
WindDir           0
TPM1              0
TPM2.5            0
TPM10             0
GeffRef           0
TempRef           0
Cycle             0
Rainfall          0
Daylight          0
FT_Voltage1    1006
FT_Voltage2    1006
DN_Voltage1    1014
DN_Voltage2    1014
dtype: int64

In [29]:
# Set FT_Voltage1, FT_Voltage2, DN_Voltage1 and DN_Voltage2 to 0 if Daylight is 0
combined_df['FT_Voltage1'] = combined_df.apply(lambda x: 0 if x['Daylight'] == 0 else x['FT_Voltage1'], axis=1)
combined_df['FT_Voltage2'] = combined_df.apply(lambda x: 0 if x['Daylight'] == 0 else x['FT_Voltage2'], axis=1)
combined_df['DN_Voltage1'] = combined_df.apply(lambda x: 0 if x['Daylight'] == 0 else x['DN_Voltage1'], axis=1)
combined_df['DN_Voltage2'] = combined_df.apply(lambda x: 0 if x['Daylight'] == 0 else x['DN_Voltage2'], axis=1)

# Check if combined_df has any NaN values
combined_df.isna().sum()

PTemp_C_Max      0
AirTC_Avg        0
RH               0
WS_ms_Avg        0
WindDir          0
TPM1             0
TPM2.5           0
TPM10            0
GeffRef          0
TempRef          0
Cycle            0
Rainfall         0
Daylight         0
FT_Voltage1    276
FT_Voltage2    276
DN_Voltage1    282
DN_Voltage2    282
dtype: int64

In [36]:
# Get the FT and DN NaN rows
# FT_NaN if either FT_Voltage1 or FT_Voltage2 is NaN
FT_NaN = combined_df[(combined_df['FT_Voltage1'].isna()) | (combined_df['FT_Voltage2'].isna())].index.values

# DN_NaN if either DN_Voltage1 or DN_Voltage2 is NaN
DN_NaN = combined_df[(combined_df['DN_Voltage1'].isna()) | (combined_df['DN_Voltage2'].isna())].index.values

In [37]:
print(len(FT_NaN), len(DN_NaN))

276 282


In [40]:
# Save FT_NaN and DN_NaN as columns to a csv file with no header
np.savetxt('FT_NaN.csv', FT_NaN, delimiter=',', fmt='%s')
np.savetxt('DN_NaN.csv', DN_NaN, delimiter=',', fmt='%s')


In [41]:
# Fill NaN values in FT_Voltage1, FT_Voltage2, DN_Voltage1 and DN_Voltage2 with the mean of the previous and next values
combined_df['FT_Voltage1'] = combined_df['FT_Voltage1'].fillna(combined_df['FT_Voltage1'].interpolate())
combined_df['FT_Voltage2'] = combined_df['FT_Voltage2'].fillna(combined_df['FT_Voltage2'].interpolate())
combined_df['DN_Voltage1'] = combined_df['DN_Voltage1'].fillna(combined_df['DN_Voltage1'].interpolate())
combined_df['DN_Voltage2'] = combined_df['DN_Voltage2'].fillna(combined_df['DN_Voltage2'].interpolate())

# Check if combined_df has any NaN values
combined_df.isna().sum()


PTemp_C_Max    0
AirTC_Avg      0
RH             0
WS_ms_Avg      0
WindDir        0
TPM1           0
TPM2.5         0
TPM10          0
GeffRef        0
TempRef        0
Cycle          0
Rainfall       0
Daylight       0
FT_Voltage1    0
FT_Voltage2    0
DN_Voltage1    0
DN_Voltage2    0
dtype: int64

In [42]:
combined_df

,PTemp_C_Max,AirTC_Avg,RH,WS_ms_Avg,WindDir,TPM1,TPM2.5,TPM10,GeffRef,TempRef,Cycle,Rainfall,Daylight,FT_Voltage1,FT_Voltage2,DN_Voltage1,DN_Voltage2
DateTime,,,,,,,,,,,,,,,,,
2022-01-10 00:00:00,14.130,14.170,70.200,0.0810,252.700,42.0,62.0,70.0,0.000559,11.0750,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-10 00:01:00,14.116,14.150,70.322,0.0742,215.844,42.0,62.0,67.0,0.001397,11.0625,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-10 00:02:00,14.102,14.130,70.444,0.0674,178.988,44.0,61.0,64.0,0.002235,11.0500,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-10 00:03:00,14.088,14.110,70.566,0.0606,142.132,45.0,64.0,68.0,0.006148,11.0250,3.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-10 00:04:00,14.074,14.090,70.688,0.0538,105.276,44.0,63.0,67.0,0.000559,10.9900,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-19 23:55:00,31.690,29.340,41.480,0.7820,156.400,37.0,55.0,61.0,0.036891,27.9250,7.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-19 23:56:00,31.646,29.482,41.006,0.6770,149.340,37.0,55.0,61.0,0.001537,27.9550,7.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-04-19 23:57:00,31.602,29.624,40.532,0.5720,142.280,33.0,49.0,53.0,0.085030,27.9800,7.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
# Save combined_df to a csv file
combined_df.to_csv('Combined_Data_with_Voltages.csv')